# **CAPITULO I: EJEMPLOS DE OPENMP**

En este capitulo veremos dos ejemplos básicos y utilizaremos las directiva **#pragma omp critical**.

**#pragma omp critical** nos sirve para rodear fragmento de codigo que se ejecuta por un solo hilo a la vez, es decir que los **p** hilos ejecutarán ese fragmento pero solo uno a la vez.

### **Potencia de dos números**
$$a^b$$
La potencia de dos números se puede hallar con la multiplicación de **a** (**b** veces):
$$a^b = a\times a\times ...\times a \textup{ (b veces)}$$
Se puede dividir esa multiplicación en **p** fragmentos donde **p** es el numero de hilos.
$$ a^b = (a\times ...\times a \textup{ (b/p veces)}) \times (a\times ...\times a \textup{ (b/p veces)}) \times ...\times (a\times ...\times a \textup{ (b/p veces)})$$ 
$$a^b = a^{b/p} \times a^{b/p} \times ...\times a^{b/p}$$

Donde cada hilo ejecutará 
$$ a^{b/p}$$

El codigo secuencial es:
```c++
int potencia(int a, int b){
    int pot = 1
    for (int i = 0; i < b; i++){
        pot *= a;
    }
    return pot;
}
```

Ya con esto podemos escribir un codigo paralelo para resolver este problema.


In [10]:
%%writefile potencia.cpp
#include <stdio.h>
#include <omp.h>

void potencia(long a, long b,long long *pot);

int main(){
    //--# variable de potencia total
    long long potTotal = 1;
    
    //--# variable que almacena el total de hilos a utilizar
    int threads = 4;
    
    //--# Declaramos los valores de a y b 
    long a = 2;
    long b = 24;
    
#pragma omp parallel num_threads(threads)
{
    potencia(a, b, &potTotal);
}
    printf("la potencia es : %lld", potTotal);
    return 0;
}

void potencia(long a, long b,long long *potTotal){
    int thread = omp_get_thread_num(); //# id del hilo
    int threads = omp_get_num_threads(); //# total de hilos
     
    int step = b/threads;
    int x = thread * step;
    int y = x + step;
    
    long potPartial = 1; //# potencia parcial
    
    for (int i = x; i < y; i++){
        potPartial *= a;
    }
#pragma omp critical
    *potTotal *= potPartial;
}


Overwriting potencia.cpp


Ejecutamos nuestros código paralelo con datos de a=2 y b=24 donde:
$$a^b=2^{24}=167772116$$

In [11]:
!g++ -fopenmp potencia.cpp
!a.exe

la potencia es : 16777216


### Suma de elementos de un arreglo unidimensional

Tenemos por ejemplo un arreglo:
<img src="array.jpg">
la suma de los elementos del array es:
$$4+5+7+2+1+3+4+12+15+11+3+8=75$$
El código secuencial es el siguiente:

```c++
int sumaSecuencial(int arr[], int size){
    int suma = 0;
    for (int i = 0; i < size; i++){
        suma += arr[i];
    }
    return suma;
}
```

La idea de paralelizar es dividir en fragmentos. por ejemplo si tenemos  **p = 3** hilos y el tamaño del arreglo **n = 12** podemos dividir el arreglo en **n/p = 4** fragmentos, tomemos el arreglo anterior y dividamos de la siguiente manera:

<img src="array1.jpg">

El hilo p = 0 sumará el fragmento amarillo con indices **0, 1, 2, 3**

El hilo p = 1 sumará el fragmento rojo con indices **4, 5, 6, 7**

El hilo p = 3 sumará el fragmento gris con indices **8, 9, 10, 11**

la suma de **p = 0** es:
$$4+5+7+2=18$$

la suma de **p = 1** es:
$$1+3+4+12=20$$

la suma de **p = 2** es:
$$15+11+3+8=37$$

y la suma total es:
$$18+20+37=75$$

Con esto ya podemos escribir el código paralelo 


In [4]:
%%writefile sumaParalela.cpp
#include <stdio.h>
#include <omp.h>

void sumaParalela(int arr[], int size,int *sumArray);

int main(){
    //--# variable de suma total
    int sumTotal = 0;
    
    //--# variable que almacena el total de hilos a utilizar
    int threads = 4;
    
    //--# Declaramos el arreglo 
    int arr[12] = {4, 5, 7, 2, 1, 3, 4, 12, 15, 11, 3, 8};
    int size = 12;
    
#pragma omp parallel num_threads(threads)
{
    sumaParalela(arr, size, &sumTotal);
}
    printf("la suma es : %d", sumTotal);
    return 0;
}

void sumaParalela(int arr[], int size,int *sumArray){
    int sumPartial = 0;
    int thread = omp_get_thread_num();
    int threads = omp_get_num_threads();
    int step = size/threads;
    int a = thread * step;
    int b = a + step;
    for (int i = a; i < b; i++){
        sumPartial += arr[i];
    }
#pragma omp critical
    *sumArray += sumPartial;
}

Writing sumaParalela.cpp


Ejecutamos nuestro código paralelo con los datos de arreglo anterior

In [5]:
!g++ -fopenmp sumaParalela.cpp
!a.exe

la suma es : 75
